<a href="https://colab.research.google.com/github/skarthikgit/bulk-email/blob/main/Fake_News_Baselines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# uncomment to install
#!pip install tensorflow-text

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls -ltr /content/drive/MyDrive/Fake_news/

total 10713
-rw------- 1 root root 10969548 Aug 29  2020 news_articles.csv


In [9]:
!ls -ltr /content/drive/MyDrive/Fake_news/news_articles.csv

-rw------- 1 root root 10969548 Aug 29  2020 /content/drive/MyDrive/Fake_news/news_articles.csv


In [10]:
pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.3 MB/s eta 0:00:00


In [11]:
#importing basic libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import optuna
from optuna.samplers import TPESampler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score


import warnings
warnings.filterwarnings("ignore")

# Suppress Optuna logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Fake_news/news_articles.csv')

In [ ]:
data.tail(10)

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.shape

###Data Preparation and Mining

In [ ]:
#check for null values
data.isnull().sum()

In [ ]:
#drop the null values
df.dropna(inplace=True)

In [ ]:
#Date Time split to see if any trend
timesplit = df['published'].str.split('T', 1, expand=True)
df['Time']=timesplit[1]
df['Date']=timesplit[0]

In [ ]:
#Split the Month and Day Data
#Since all our data in from 2016, we might be able to see a trend if any, through the months
datesplit = df['Date'].str.split('-',n=-1, expand=True)
df['Year']=datesplit[0]
df['Month']=datesplit[1]
df['Day']=datesplit[2]

In [ ]:
#drop some error producing rows
df.loc[df.Month == 'content/uploads/2014/04/jucundus']
df.drop(848, inplace=True)
df.loc[df.Month == 'content/uploads/2015/07/Earth']
df.drop(1838, inplace=True)

In [ ]:
#Change the type for ease of plotting
df = df.astype({"Month": float, "Day":float})

In [ ]:
df.tail(5)
#cleaner and transformed data

In [ ]:
#Labelize the label column
#Create Dummies to represent Real and Fake by 1 and 0
dum_type=pd.get_dummies(df.label,drop_first=True,prefix="")
df=df.join(dum_type)
df.drop('label', axis=1, inplace=True)
df.rename(columns={'_Real':'Real'}, inplace=True)

###EDA

In [ ]:
#plot to see Counts of Real and Fake data
sb.set_theme(style="whitegrid")
ax = sb.countplot(x="Real", data=df, palette='Set3')
ax.set_title("Count of Real and Fake Data")

In [ ]:
#plot to see Counts of Language data
sb.set_theme(style="whitegrid")
ax = sb.countplot(x="language", data=df, palette='Set3')
ax.set_title("Count of Languages of the news")

In [ ]:
#plot to see Counts of type of news data
sb.set_theme(style="whitegrid")
ax = sb.countplot(x="type", data=df, palette='Set3')
ax.set_title("Count of Type of the news")

In [ ]:
#plot to see Counts news data that have images or no
sb.set_theme(style="whitegrid")
ax = sb.countplot(x="hasImage", data=df, palette='Set3')
ax.set_title("Count of news with Images")

In [ ]:
#plot to see top 10 Authors with most news articles
sb.set_theme(style="whitegrid")
ax = sb.countplot(y="author", data=df, palette='Set3', order =df.author.value_counts().iloc[:10].index)
ax.set_title("Top 10 Authors with count of news")

In [ ]:
#plot to see top 10 Authors with most news urls
sb.set_theme(style="whitegrid")
ax = sb.countplot(y="site_url", data=df, palette='Set3', order =df.site_url.value_counts().iloc[:10].index)
ax.set_title("Top 10 Site URLS with count of news")

In [ ]:
#plot to see Months of pblished news
sb.set_theme(style="whitegrid")
ax = sb.countplot(x="Month", data=df, palette='Set3')
ax.set_title("Months of published news")

In [ ]:
#plot to see Days of pblished news
sb.set_theme(style="whitegrid")
ax = sb.countplot(x="Day", data=df, palette='Set3')
sb.set(rc = {'figure.figsize':(10,10)})
ax.set_title("Days of published news")

In [ ]:
#Fake Authors
df.loc[df.Real == 0].author.value_counts().head(10)

In [ ]:
#Fake URL sites
df.loc[df.Real == 0].site_url.value_counts().head(10)

In [ ]:
#Real Authors
df.loc[df.Real == 1].author.value_counts().head(10)

In [ ]:
#Real URL sites
df.loc[df.Real == 1].site_url.value_counts().head(10)

In [ ]:
real = set(df[df['Real'] == 1]['site_url'].unique())
fake = set(df[df['Real'] == 0]['site_url'].unique())
print(f"{real & fake}")

###Feature Engineering

In [ ]:
#target is our column to be predicted and features are the columns that we will use for modelling
target = df.Real
features = df[['author','site_url','text_without_stopwords']]

In [ ]:
features

In [ ]:
#since all the feature columns are text data, we want to use text mining techniques, we put all of them in the same column for ease of processing
features['combined_url_text'] = features["author"]+" "+features["site_url"] + " " + features["text_without_stopwords"]
features.drop(['author','site_url', 'text_without_stopwords'], axis = 1, inplace = True)
features=features.combined_url_text

In [ ]:
features

###TFID Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer_words = TfidfVectorizer(stop_words='english')
X = vectorizer_words.fit_transform(features)

In [ ]:
#trying to cluster the 2 data centres ie fake and real news
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 2, init = 'k-means++', max_iter = 100, n_init = 1, verbose = True)
km.fit(X)

In [ ]:
import numpy as np
np.unique(km.labels_, return_counts=True)
features.reset_index(drop=True, inplace=True)

In [ ]:
#Here we are collecting all the fake data into one cluster and all the real data into another.
#We are doing this as a pre step to the coming logic.
text={}
for i,cluster in enumerate(km.labels_):
    oneDocument = features[i]
    if cluster not in text.keys():
        text[cluster] = oneDocument
    else:
        text[cluster] += oneDocument

In [ ]:
#importing NLTK libraries for NLP techiniques
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from string import punctuation
from heapq import nlargest
import nltk

In [ ]:
#here we are finding the keywords that appear in both the clusters and we do not want to include the stop words as those will be the most frequent
_stopwords = set(stopwords.words('english') + list(punctuation))

keywords = {}
counts={}
for cluster in range(2):
    word_sent = word_tokenize(text[cluster].lower())
    word_sent=[word for word in word_sent if word not in _stopwords]
    freq = FreqDist(word_sent)
    keywords[cluster] = nlargest(100, freq, key=freq.get)
    counts[cluster]=freq

In [ ]:
keywords

In [ ]:
#here we are finding keyword that are uniques to each cluster
set0 = set(keywords[0])
set1 = set(keywords[1])
unique0=set0-set1
unique1 =set1-set0

In [ ]:
unique_keys={}
unique_keys[0]=nlargest(10, unique0, key=counts[0].get)
unique_keys[1]=nlargest(10, unique1, key=counts[1].get)

In [ ]:
unique_keys

###Modelling and Prediction

In [ ]:
#using the Vectorizer before doing the classification
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(features,target,test_size=0.20)

vectorizer = TfidfVectorizer(stop_words = 'english')
tfidf_train = vectorizer.fit_transform(X_train)
tfidf_test = vectorizer.transform(X_test)
tfidf_df = pd.DataFrame(tfidf_train.A, columns=vectorizer.get_feature_names())

In [ ]:
print(tfidf_train.shape)
print(y_train.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

# Initialize the KNeighborsClassifier with n_neighbors=5
KNb = KNeighborsClassifier(n_neighbors=5)

# Train the classifier on the training data
KNb.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = KNb.predict(tfidf_test)

# Calculate the accuracy score
KNscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print(f"Accuracy Score: {KNscore}")
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

# Initialize the RandomForestClassifier with n_estimators=1000
RF = RandomForestClassifier(n_estimators=1000)

# Train the classifier on the training data
RF.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = RF.predict(tfidf_test)

# Calculate the accuracy score
RFscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("Random Forest Model accuracy: %0.4f" % RFscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Initialize the AdaBoostClassifier with a base DecisionTreeClassifier (max_depth=10) and n_estimators=5
Adab = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=5)

# Train the classifier on the training data
Adab.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = Adab.predict(tfidf_test)

# Calculate the accuracy score
ABscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("AdaBoost Classifier accuracy: %0.4f" % ABscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

# Initialize the LogisticRegression model
LR = LogisticRegression(max_iter=1000)

# Train the classifier on the training data
LR.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = LR.predict(tfidf_test)

# Calculate the accuracy score
LRscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("Logistic Regression accuracy: %0.4f" % LRscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics

# Initialize the Support Vector Machine model
SVM = SVC(kernel='linear')

# Train the classifier on the training data
SVM.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = SVM.predict(tfidf_test)

# Calculate the accuracy score
SVMscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("Support Vector Machine accuracy: %0.4f" % SVMscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Initialize the Naive Bayes model
NB = MultinomialNB()

# Train the classifier on the training data
NB.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = NB.predict(tfidf_test)

# Calculate the accuracy score
NBscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("Naive Bayes accuracy: %0.4f" % NBscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics

# Initialize the DecisionTreeClassifier model
DT = DecisionTreeClassifier(max_depth=10)

# Train the classifier on the training data
DT.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = DT.predict(tfidf_test)

# Calculate the accuracy score
DTscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("Decision Tree accuracy: %0.4f" % DTscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

# Initialize the MLPClassifier model
MLP = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000)

# Train the classifier on the training data
MLP.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = MLP.predict(tfidf_test)

# Calculate the accuracy score
MLPscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score and the classification report
print("MultiLayer Perceptron accuracy: %0.4f" % MLPscore)
print("Classification Report:")
print(classification_report)

###GridSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# Initialize the LogisticRegression model
LR = LogisticRegression(max_iter=1000)

# Define the hyperparameter grid
param_grid = {'C': [0.1, 1, 10]}

# Initialize GridSearchCV with 2-fold cross-validation
grid_search = GridSearchCV(LR, param_grid, cv=2)

# Train the model with GridSearchCV
grid_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = grid_search.predict(tfidf_test)

# Calculate the accuracy score
LRscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("Logistic Regression best parameters:", grid_search.best_params_)
print("Logistic Regression accuracy: %0.4f" % LRscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# Initialize the SVC model
SVM = SVC()

# Define the hyperparameter grid
param_grid = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10]}

# Initialize GridSearchCV with 2-fold cross-validation
grid_search = GridSearchCV(SVM, param_grid, cv=2)

# Train the model with GridSearchCV
grid_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = grid_search.predict(tfidf_test)

# Calculate the accuracy score
SVMscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("SVM best parameters:", grid_search.best_params_)
print("SVM accuracy: %0.4f" % SVMscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# Initialize the Naive Bayes model
NB = MultinomialNB()

# Define the hyperparameter grid
param_grid = {'alpha': [0.1, 1, 10]}

# Initialize GridSearchCV with 2-fold cross-validation
grid_search = GridSearchCV(NB, param_grid, cv=2)

# Train the model with GridSearchCV
grid_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = grid_search.predict(tfidf_test)

# Calculate the accuracy score
NBscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("Naive Bayes best parameters:", grid_search.best_params_)
print("Naive Bayes accuracy: %0.4f" % NBscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# Initialize the DecisionTreeClassifier model
DT = DecisionTreeClassifier()

# Define the hyperparameter grid
param_grid = {'max_depth': [5, 10, 15], 'min_samples_split': [2, 10]}

# Initialize GridSearchCV with 2-fold cross-validation
grid_search = GridSearchCV(DT, param_grid, cv=2)

# Train the model with GridSearchCV
grid_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = grid_search.predict(tfidf_test)

# Calculate the accuracy score
DTscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("Decision Tree best parameters:", grid_search.best_params_)
print("Decision Tree accuracy: %0.4f" % DTscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import metrics

# Initialize the MLPClassifier model
MLP = MLPClassifier(max_iter=1000)

# Define the hyperparameter grid
param_grid = {'hidden_layer_sizes': [(50,), (100,), (100, 100)], 'activation': ['relu', 'tanh']}

# Initialize GridSearchCV with 2-fold cross-validation
grid_search = GridSearchCV(MLP, param_grid, cv=2)

# Train the model with GridSearchCV
grid_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = grid_search.predict(tfidf_test)

# Calculate the accuracy score
MLPscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("MultiLayer Perceptron best parameters:", grid_search.best_params_)
print("MultiLayer Perceptron accuracy: %0.4f" % MLPscore)
print("Classification Report:")
print(classification_report)

###RandomizedSearchCV

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

# Initialize the LogisticRegression model
LR = LogisticRegression(max_iter=1000)

# Define the hyperparameter distribution
param_dist = {'C': [0.1, 1, 10, 100], 'solver': ['liblinear', 'saga']}

# Initialize RandomizedSearchCV with 2-fold cross-validation
random_search = RandomizedSearchCV(LR, param_dist, n_iter=10, cv=2)

# Train the model with RandomizedSearchCV
random_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = random_search.predict(tfidf_test)

# Calculate the accuracy score
LRscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("Logistic Regression best parameters:", random_search.best_params_)
print("Logistic Regression accuracy: %0.4f" % LRscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

# Initialize the SVC model
SVM = SVC()

# Define the hyperparameter distribution
param_dist = {'kernel': ['linear', 'rbf'], 'C': [0.1, 1, 10, 100], 'gamma': ['scale', 'auto']}

# Initialize RandomizedSearchCV with 2-fold cross-validation
random_search = RandomizedSearchCV(SVM, param_dist, n_iter=10, cv=2)

# Train the model with RandomizedSearchCV
random_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = random_search.predict(tfidf_test)

# Calculate the accuracy score
SVMscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("SVM best parameters:", random_search.best_params_)
print("SVM accuracy: %0.4f" % SVMscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

# Initialize the Naive Bayes model
NB = MultinomialNB()

# Define the hyperparameter distribution
param_dist = {'alpha': [0.1, 0.5, 1, 2, 10]}

# Initialize RandomizedSearchCV with 2-fold cross-validation
random_search = RandomizedSearchCV(NB, param_dist, n_iter=5, cv=2)

# Train the model with RandomizedSearchCV
random_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = random_search.predict(tfidf_test)

# Calculate the accuracy score
NBscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("Naive Bayes best parameters:", random_search.best_params_)
print("Naive Bayes accuracy: %0.4f" % NBscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

# Initialize the DecisionTreeClassifier model
DT = DecisionTreeClassifier()

# Define the hyperparameter distribution
param_dist = {'max_depth': [5, 10, 15, 20], 'min_samples_split': [2, 10, 20]}

# Initialize RandomizedSearchCV with 2-fold cross-validation
random_search = RandomizedSearchCV(DT, param_dist, n_iter=10, cv=2)

# Train the model with RandomizedSearchCV
random_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = random_search.predict(tfidf_test)

# Calculate the accuracy score
DTscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("Decision Tree best parameters:", random_search.best_params_)
print("Decision Tree accuracy: %0.4f" % DTscore)
print("Classification Report:")
print(classification_report)

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn import metrics

# Initialize the MLPClassifier model
MLP = MLPClassifier(max_iter=1000)

# Define the hyperparameter distribution
param_dist = {'hidden_layer_sizes': [(50,), (100,), (100, 100)], 'activation': ['relu', 'tanh']}

# Initialize RandomizedSearchCV with 2-fold cross-validation
random_search = RandomizedSearchCV(MLP, param_dist, n_iter=5, cv=2)

# Train the model with RandomizedSearchCV
random_search.fit(tfidf_train, y_train)

# Predict the labels for the test data
y_pred = random_search.predict(tfidf_test)

# Calculate the accuracy score
MLPscore = metrics.accuracy_score(y_test, y_pred)

# Generate the classification report
classification_report = metrics.classification_report(y_test, y_pred)

# Print the accuracy score, best parameters, and classification report
print("MultiLayer Perceptron best parameters:", random_search.best_params_)
print("MultiLayer Perceptron accuracy: %0.4f" % MLPscore)
print("Classification Report:")
print(classification_report)